# Proof of Concept : Basic chatbot implementation for BU Law Project

- Upload the chunked data into ChromaDB (Ensure chroma_db folder exists with vectorized legal documents).

- Run the notebook step-by-step.

In [1]:
!pip install sentence-transformers chromadb gradio smolagents requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta

In [8]:
import chromadb
import gradio as gr
import requests
from sentence_transformers import SentenceTransformer
# from smolagents import Agent

Convert User Query to Embeddings :

- Use Sentence-Transformers to generate embeddings for user queries.
Ensure that these embeddings match the ones used during document chunking.

- Use Sentence-Transformers to generate embeddings for user queries.
Ensure that these embeddings match the ones used during document chunking.

In [19]:
#!pip show smolagents

In [20]:
# import smolagents
# print(dir(smolagents))

In [21]:
# from smolagents import ToolCallingAgent


In [3]:
# Load the sentence transformer model for embeddings
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to convert a query to an embedding
def embed_query(query: str):
    return embedding_model.encode(query).tolist()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Connect to ChromaDB and Retrieve Relevant Documents :     
- Perform Similarity Search in ChromaDB

- Fetch top-k relevant documents based on cosine similarity

In [4]:
# Connect to ChromaDB where legal documents are stored
client = chromadb.PersistentClient(path="./chroma_db")  # Ensure the path contains the vectorized DB
collection = client.get_or_create_collection("legal_documents")

# Function to retrieve similar legal documents based on query
def retrieve_documents(query: str, top_k=5):
    query_embedding = embed_query(query) # Convert query to vector
    results = collection.query(
        query_embeddings=[query_embedding], # Search the database using this embedding
        n_results=top_k # Fetch top-k most similar documents
    )
    return results["documents"]  # Returns retrieved document texts


Generate AI Responses using DeepSeek-R1 :     
- Integrate with DeepSeek-R1 for Response Generation

- Once relevant documents are retrieved, they need to be processed into a meaningful response.

- Pass Retrieved Documents to DeepSeek-R1

- We send retrieved context to DeepSeek-R1, which will generate responses based on legal context.


In [5]:
# API Endpoint for DeepSeek-R1 (Update with actual endpoint)
DEEPSEEK_R1_URL = "http://localhost:5000/generate"

# Function to generate response based on retrieved documents
def generate_response(query, context):
    payload = {
        "query": query,
        "context": "\n".join(context)  # Combine retrieved document sections
    }
    response = requests.post(DEEPSEEK_R1_URL, json=payload)
    return response.json().get("generated_text", "No response generated.")

Implement an AI Agent (Qwen) with SmolAgents :     

- Implement Agentic Behavior with SmolAgents
- To enhance our chatbot’s autonomy, implement SmolAgents to Handle task delegation and Decide whether to answer directly or retrieve additional info.

- Define the AI Agent
- Use Qwen as the primary agent that determines whether to Answer directly or
Call the RAG pipeline.


In [18]:
# from smolagents import ToolCallingAgent, Tool

# # Define a dummy tool (this is required by smolagents)
# class RetrievalTool(Tool):
#     def __call__(self, query):
#         documents = retrieve_documents(query)
#         if not documents:
#             return "No relevant legal information found. Please refine your query."
#         return generate_response(query, documents)

# # Define the chatbot agent using ToolCallingAgent
# class LegalChatbotAgent(ToolCallingAgent):
#     def __init__(self):
#         super().__init__(
#             tools={"retrieval": RetrievalTool()},  # Pass required tools
#             model=None  # Set to None since we don't need a separate LLM model here
#         )

#     def decide_action(self, query):
#         if len(query.split()) < 5:
#             return "direct_answer"
#         return "use_rag"

#     def run(self, query):
#         action = self.decide_action(query)
#         if action == "direct_answer":
#             return "This is a general legal question. Please provide more details."
#         else:
#             return self.tools["retrieval"](query)  # Use the retrieval tool

# # Instantiate the chatbot agent
# chatbot = LegalChatbotAgent()


 Deploy as a Chatbot with Gradio :
   
 - Once our model is working, integrate it into Gradio to make it accessible

In [7]:
# Define Gradio Interface
def chatbot_ui(query):
    return chatbot.run(query)

# Create a simple chatbot interface
iface = gr.Interface(
    fn=chatbot_ui,
    inputs="text",
    outputs="text",
    title="Legal Chatbot",
    description="Ask any legal question related to eviction and housing rights."
)

# Launch the chatbot
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://adc206b58040dd7f43.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
